In [1]:
# Part 1
# Andrew French - 11147452

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.datasets import load_iris, load_breast_cancer, load_digits
from sklearn import tree
from sklearn.linear_model import LinearRegression, LogisticRegression

def part1():

    datasets = {"iris": load_iris(), "breast cancer":load_breast_cancer(), "digits": load_digits()}

    print("-------------------------------------------")

    for key in datasets.keys():
        dataset = datasets[key]

        print("Training the", key, "dataset...")

        # Split the data in training and testing sets
        print("Test = 40% - Train = 60%")
        X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.4, random_state=0)

        #Loop with different depths - 1 to 10
        dataframe = pd.DataFrame(columns=['Depth', 'Avg # of Nodes', 'Accuracy'])
        clf = tree.DecisionTreeClassifier()
        for depth in range(1, 16):

            clf = tree.DecisionTreeClassifier(max_depth=depth)
            clf = clf.fit(X_train, y_train)
            scores = cross_val_score(clf, dataset.data, dataset.target, cv=8)
            #Unsure if the avg # of nodes is correct
            dataframe.loc[depth] = [clf.get_depth(), clf.tree_.node_count/8, "%0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)]

        print(dataframe)

        # Select the best value of hyper-parameter and then use the entire training data to build a classifier. 
        best_value = clf.get_depth()
        print("The best value for the depth hyper-parameter is", best_value)

        # Report the number of nodes and the accuracy of the classifier on the test set.
        print("Testing the", key, "dataset with test data and with hyper-parameter...")
        clf = tree.DecisionTreeClassifier(max_depth=best_value)
        clf = clf.fit(X_test, y_test)
        scores = cross_val_score(clf, dataset.data, dataset.target, cv=8)
        # Again not sure if number of nodes is correct
        print("Number of nodes:", clf.tree_.node_count/8)
        print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

        print("---------------------------------------------------")

# Part 2

def part2():

    X, y = load_iris(return_X_y=True)
    clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X, y)
    print(clf.predict(X[:2, :]))

    print(clf.predict_proba(X[:2, :]))

    print(clf.score(X, y))
    
#     Construct the problem by defining a function (formula) that decides the
#     class label of a point in R
#     . This is the true f which is typically unknown
#     but not here since you are defining the problem and generating the data.
#     You must then generate some training and test data in R
#     2×{P, N}, where
#     P and N indicate the positive and negative class respectively. Visualise a
#     sample of points.
#     • Show how various tree sizes (from shallow to deep) perform with different
#     training sizes. Iterate over a range tree depths and training data sizes and
#     report the test accuracy in a table.
#     • Provide a visualization of the actual and learned class boundaries (for a
#     single tree). The visualisation must demonstrate the issue.
#     • Very briefly (in a few sentences) discuss what would happen if the problem
#     was R
#     d with a large d.
    
    
    
# Part 3

def part3():
    # Reading the file, latin encoding is used to avoid some errors
    data = pd.read_csv('assignment1-2019-data.csv',encoding=('latin'),quoting=3)

    # Creating the list of input and output variables.
    inputVariables=list(data)
    del inputVariables[4]
    outputVariables=list(data)[4]
    inputData=data[inputVariables]
    
    for column in inputData.columns:
        if inputData[column].dtype==object:
            dummyCols=pd.get_dummies(inputData[column])
            inputData=inputData.join(dummyCols)
            del inputData[column]
    
    model_1=LinearRegression()
    model_1.fit(inputData, data[outputVariables])
    coefficients=pd.DataFrame({'name':list(inputData),'value':model_1})
    print("Mean squared error:", np.mean((model_1.predict(inputData) - data[outputVariables]) ** 2))
    print("R²:", model_1.score(inputData, data[outputVariables]))
    
#     Round the coefficients of the learned model to integers and write a casebased definition of the functions identified by the regression model(s).
#     See http://scikit-learn.org/stable/modules/linear_model.html or if you are familiar
#     with R see https://www.statsmodels.org/dev/example_formulas.html
#     Each case must be a linear expression of the input variables (one expression
#     per level of the categorical variable). For example:
#     y =
#     
#     
#     
#     w1,1x1 + w1,2x2 + w1,3x3 + b1, c = l1
#     w2,1x1 + w2,2x2 + w2,3x3 + b2, c = l2
#     w3,1x1 + w3,2x2 + w3,3x3 + b3, c = l3
#     where wi,j is the integer coefficient of the j-th numerical feature corresponding to the i-th level in the domain of the categorical feature; bi
#     is
#     the intercept in each case, and li
#     is the i-th level in the domain of the
#     categorical feature c.
#     • Depending on the method you used for regression, described how a new
#     (unseen) data point (an input vector of size 4) can be assigned a predicted
#     value.

#Current Running Part
part2()
    

[0 0]
[[9.81802911e-01 1.81970751e-02 1.43580537e-08]
 [9.71729527e-01 2.82704429e-02 3.00353141e-08]]
0.9733333333333334


/Users/andyfrench/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
